# GRU  FOR TIME SERIES PREDICTION @@@@
# STOCK MARKET DATASET


#MODEL :

#1.GRU
#2.GRU
#3.FULL-CONNECTED (CLASSICATION LAYER)


In [1]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

#from google.cloud import bigquery
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, DenseFeatures,
                                     Conv1D, MaxPool1D,
                                     Reshape, RNN,
                                     LSTM, GRU, Bidirectional)
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# To plot pretty figures
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# For reproducible results.
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

In [3]:
STOCK_HISTORY_COLUMN = 'close_values_prior_260'
COL_NAMES = ['day_' + str(day) for day in range(0, 260)]
LABEL = 'direction_numeric'

In [4]:
N_TIME_STEPS = 260
N_LABELS = 3

Xtrain = pd.read_csv('stock-train.csv')
Xvalid = pd.read_csv('stock-valid.csv')

ytrain = Xtrain.pop(LABEL)
yvalid  = Xvalid.pop(LABEL)

ytrain_categorical = to_categorical(ytrain.values)
yvalid_categorical = to_categorical(yvalid.values)

In [5]:
def plot_curves(train_data, val_data, label='Accuracy'):
    """Plot training and validation metrics on single axis.

    Args:
        train_data: list, metrics obtrained from training data.
        val_data: list, metrics obtained from validation data.
        label: str, title and label for plot.
    Returns:
        Matplotlib plot.
    """
    plt.plot(np.arange(len(train_data)) + 0.5,
             train_data,
             "b.-", label="Training " + label)
    plt.plot(np.arange(len(val_data)) + 1,
             val_data, "r.-",
             label="Validation " + label)
    plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    plt.legend(fontsize=14)
    plt.xlabel("Epochs")
    plt.ylabel(label)
    plt.grid(True)  

In [7]:
rnn_hidden_units = [N_TIME_STEPS // 4,
                    N_TIME_STEPS // 8]

model = Sequential()

# Reshape inputs to pass through RNN layer.
model.add(Reshape(target_shape=[N_TIME_STEPS, 1]))

for layer in rnn_hidden_units[:-1]:
    model.add(GRU(units=layer,
                  activation='relu',
                  return_sequences=True))

model.add(GRU(units=rnn_hidden_units[-1],
              return_sequences=False))
model.add(Dense(units=N_LABELS,
                activation="softmax",
                kernel_regularizer=tf.keras.regularizers.l1(l=0.1)))

model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x=Xtrain.values,
                    y=ytrain_categorical,
                    batch_size=Xtrain.shape[0],
                    validation_data=(Xvalid.values, yvalid_categorical),
                    epochs=50,
                    verbose=1)

Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 3.0114 - accuracy: 0.3645 - val_loss: 3.0027 - val_accuracy: 0.3475
Epoch 2/50
1/1 [==============================] - 2s 2s/step - loss: 2.9999 - accuracy: 0.3633 - val_loss: 2.9932 - val_accuracy: 0.3400
Epoch 3/50
1/1 [==============================] - 2s 2s/step - loss: 2.9888 - accuracy: 0.3611 - val_loss: 2.9838 - val_accuracy: 0.3312
Epoch 4/50
1/1 [==============================] - 2s 2s/step - loss: 2.9781 - accuracy: 0.3603 - val_loss: 2.9745 - val_accuracy: 0.3296
Epoch 5/50
1/1 [==============================] - 2s 2s/step - loss: 2.9678 - accuracy: 0.3589 - val_loss: 2.9653 - val_accuracy: 0.3279
Epoch 6/50
1/1 [==============================] - 2s 2s/step - loss: 2.9577 - accuracy: 0.3577 - val_loss: 2.9560 - val_accuracy: 0.3283
Epoch 7/50
1/1 [==============================] - 2s 2s/step - loss: 2.9478 - accuracy: 0.3585 - val_loss: 2.9465 - val_accuracy: 0.3363
Epoch 8/50
1/1 [=========================